In [1]:
import numpy as np
import cv2
import base64
import requests
import os
import pickle

---------------------------------
Model Convert Step

In [2]:
url_hoggen = 'http://localhost:8000/api/genhog'
cars_path = f"Plants_dataset/"
written_path = f'plants_models/'

In [3]:
def saveModel(obj, name):
    model = f'{written_path}{name}.pk'
    pickle.dump(obj, open(model, 'wb'))
    print(f'saved {model} successfully.')

def loadModel(name):
    return pickle.load(open(f'{written_path}{name}', 'rb'))

In [12]:
def getHOGData(img):
    # convert image to buffer
    v, buffer = cv2.imencode('.jpg', img)
    # convert buffer of image to base64
    img_str = "img,"+str.split(str(base64.b64encode(buffer)), "'")[1]

    req = requests.get(url_hoggen, json={"img":img_str})
    return req.json()['data']

type_dict = {
}

In [13]:
#convert cars dataset into HOG Data & save file
def convertAllAndSave(train_test=True, returned=False):
  hog_list = []
  type_list = []

  dest_path = 'train' if train_test else 'test'
  
  print(f"Converting {dest_path} plants into HOG Model...")
  i = 0
  for subFolder in os.listdir(cars_path+dest_path):
      # print(f'{i}. {subFolder}')
      type_dict[subFolder] = i
      try:
          subFolderRelPath = os.path.join(cars_path+dest_path,subFolder)
          for file in os.listdir(subFolderRelPath):
            img = cv2.imread(subFolderRelPath+"/"+file)
            hog_list.append(getHOGData(img))
            type_list.append(type_dict[subFolder])
          
          i = i + 1
      except:
          print("error in dir: " + subFolder)

  print(f'\n{len(hog_list)} Model loaded complete!')
  saveModel(hog_list, ('' if train_test else 'test_') + 'plantHog')
  saveModel(type_list, ('' if train_test else 'test_') + 'plantType')

  if returned:
    return {"hog": hog_list, "type": type_list}
  else:
    return None

In [11]:

hog_list = []
type_list = []
dest_path = 'train'

type_dict = {
}

i = 0
for subFolder in os.listdir(cars_path+dest_path):
    print(f'{i}. {subFolder}')
    type_dict[subFolder] = i
    try:
        subFolderRelPath = os.path.join(cars_path+dest_path,subFolder)
        # for file in os.listdir(subFolderRelPath):
        #   img = cv2.imread(subFolderRelPath+"/"+file)
        #   hog_list.append(getHOGData(img))
          # type_list.append(type_dict[subFolder])
        
        i = i + 1
    except:
        print("error in dir: " + subFolder)

0. kale
1. sweetpotatoes
2. spinach
3. peperchili
4. classname.txt
5. tobacco
6. guava
7. bilimbi
8. aloevera
9. cucumber
10. pomelo
11. watermelon
12. cantaloupe
13. waterapple
14. curcuma
15. mango
16. coconut
17. galangal
18. eggplant
19. cassava
20. corn
21. soybeans
22. banana
23. longbeans
24. ginger
25. paddy
26. papaya
27. shallot
28. pineapple
29. melon
30. orange


Convert Model 

In [9]:
convertAllAndSave(True)

Converting train plants into HOG Model...
0. kale
error in dir: kale
0. sweetpotatoes
error in dir: sweetpotatoes
0. spinach
error in dir: spinach
0. peperchili
error in dir: peperchili
0. classname.txt
error in dir: classname.txt
0. tobacco
error in dir: tobacco
0. guava
error in dir: guava
0. bilimbi
error in dir: bilimbi
0. aloevera
error in dir: aloevera
0. cucumber
error in dir: cucumber
0. pomelo
error in dir: pomelo
0. watermelon
error in dir: watermelon
0. cantaloupe
error in dir: cantaloupe
0. waterapple
error in dir: waterapple
0. curcuma
error in dir: curcuma
0. mango
error in dir: mango
0. coconut
error in dir: coconut
0. galangal
error in dir: galangal
0. eggplant
error in dir: eggplant
0. cassava
error in dir: cassava
0. corn
error in dir: corn
0. soybeans
error in dir: soybeans
0. banana
error in dir: banana
0. longbeans
error in dir: longbeans
0. ginger
error in dir: ginger
0. paddy
error in dir: paddy
0. papaya
error in dir: papaya
0. shallot
error in dir: shallot
0. p

FileNotFoundError: [Errno 2] No such file or directory: 'plants_models/plantHog.pk'

---------------------------------
Generate Model Step

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [9]:
hog_data = loadModel('carHog.pk')
type_data = loadModel('carType.pk')

X_train = np.array(hog_data)
Y_train = np.array(type_data)


In [10]:
clf = DecisionTreeClassifier()
clf.fit(X_train, Y_train)

DecisionTreeClassifier()

----------------------
test session

In [12]:
hog_data = convertAllAndSave(False, True)
test_hog_list = hog_data['hog']
test_type_list = hog_data['type']

Converting test cars into HOG Model...
0. Tata Safari
1. .DS_Store
error in dir: .DS_Store
1. Audi
2. Toyota Innova
3. Swift
4. Hyundai Creta
5. Rolls Royce
6. Mahindra Scorpio

813 Model loaded complete!
saved car_models/test_carHog.pk successfully.
saved car_models/test_carType.pk successfully.


In [14]:
hogs_np = np.array(test_hog_list)
type_np = np.array([[label] for label in test_type_list]) #make value inside into 2D array ([1,3,2,3] => [[1],[3],[2],[3]])
X_test = hogs_np
Y_test = type_np

In [18]:
X_train, X_test, y_train, y_test = train_test_split(hogs_np, type_np, test_size=0.2, random_state=42)
# Make predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)*100
print("Accuracy:", accuracy)
print("Confusion Matrix:", accuracy)
confusion_matrix(y_test, y_pred)

Accuracy: 45.39877300613497
Confusion Matrix: 45.39877300613497


array([[11,  4,  1,  4,  3,  5,  3],
       [ 0,  7,  1,  0,  2,  1,  4],
       [ 1,  3,  6,  1,  4,  2,  1],
       [ 4,  1,  2,  3,  2,  1,  2],
       [ 4,  0,  0,  2, 15,  1,  2],
       [ 5,  0,  0,  1,  2, 13,  4],
       [ 4,  5,  1,  3,  1,  2, 19]])

In [19]:
saveModel(clf, 'carsModel')

saved car_models/carsModel.pk successfully.
